In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

def create_seq(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)
	
# split a univariate dataset into train/test sets
def split_dataset(data,steps):
	# split into standard weeks

	# restructure into windows of weekly data
	train = np.array(np.split(data, len(data)/steps))

	return train

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			x_input = data[in_start:in_end, 0]
			x_input = x_input.reshape((len(x_input), 1))
			X.append(x_input)
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return np.array(X), np.array(y)

In [3]:
root_dir = "./"

loc_2019_WEP = root_dir + "2019_full_data.csv"
loc_2020_WEP= root_dir + "2020_WEP_ONLY.csv"

#load dataset

df_2019=pd.read_csv(loc_2019_WEP, index_col=0, parse_dates=True, usecols=['DATE','WEP ($/MWh)'])
df_2020=pd.read_csv(loc_2020_WEP, index_col=0, parse_dates=True)

In [4]:
x = split_dataset(df_2019.values, 48)

In [5]:
data = x.reshape((x.shape[0]*x.shape[1],x.shape[2]))

In [6]:
train_x, train_y = to_supervised(x,336,48)

In [ ]:
#train model
# prepare data
#train_x, train_y = to_supervised(train, n_input)
# define parameters
verbose, epochs, batch_size = 1, 1000, 42
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs))
model.compile(loss='mse', optimizer='adam')
# fit network
history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
model.save(root_dir + "model_LSTM_1000.h5")

Train on 17137 samples
Epoch 1/1000
17137/17137 [==============================] - 114s 7ms/sample - loss: 6064275610.5622
Epoch 2/1000
17137/17137 [==============================] - 112s 7ms/sample - loss: 12274.2980
Epoch 3/1000
17137/17137 [==============================] - 119s 7ms/sample - loss: 12270.4543
Epoch 4/1000
17137/17137 [==============================] - 143s 8ms/sample - loss: 12265.7279
Epoch 5/1000
17137/17137 [==============================] - 126s 7ms/sample - loss: 12260.1397
Epoch 6/1000
17137/17137 [==============================] - 124s 7ms/sample - loss: 12253.6336
Epoch 7/1000
17137/17137 [==============================] - 113s 7ms/sample - loss: 12246.1187
Epoch 8/1000
17137/17137 [==============================] - 112s 7ms/sample - loss: 12237.4295
Epoch 9/1000
17137/17137 [==============================] - 113s 7ms/sample - loss: 12227.5235
Epoch 10/1000
17137/17137 [==============================] - 114s 7ms/sample - loss: 12216.1442
Epoch 11/1000
17137/1

 4956/17137 [=======>......................] - ETA: 1:38 - loss: 7200.5913

In [ ]:
#forecasting

test = split_dataset(df_2020.values, 48)
test_x, test_y = to_supervised(test,336,1)

In [ ]:
asd = model.predict(test_x[20].reshape((1,len(test_x[1]),1)))

In [ ]:
plt.plot(asd[0])
plt.plot(test_y)
test_x.shape